In [112]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


In [113]:
import torch

print(torch.__version__)
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0)) 

2.6.0.dev20241211+cu126
True
NVIDIA GeForce RTX 4060 Laptop GPU


In [114]:
model_name = "./Llama-3.2-1B-Instruct"

In [115]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")


In [134]:
input_text = "Does Aneesh know NextJs?"

In [135]:
prompt = f"""
You MUST respond with a single word: either "Yes" or "No". AND keywords no sentences.
Do NOT include any other text, explanations, or punctuation.

Determine if the following text describes a professional skill, experience, educational achievement, or qualification typically listed on a resume or CV,
those skills maybe related to a specific job or industry, or they may be general skills that are applicable to many jobs. define that as the keyword.

Examples:
1. Text: "Bachelor of Science in Computer Science from XYZ University."
   Response: Yes

2. Text: "The weather is sunny today."
   Response: No

3. Text: "Developed and deployed a machine learning model using Python and TensorFlow, resulting in a 15% improvement in prediction accuracy."
   Response: Yes

4. Text: "Proficient in Microsoft Office Suite."
   Response: Yes

5. Text: "Won the local pie-eating contest."
   Response: No

Now classify the following text:

Text: "{input_text}"
Response:"""

In [136]:
prompt = f"""
You MUST respond with ONE of the following categories ONLY: "Technical Skills", "Experience", "Project", "Education", or "Not Related".
YOU MUST also INCLUDE 0 or more than it keywords.
Do NOT include any other text, explanations, or punctuation.

Your task is to categorize text snippets based on their relevance to a professional resume or CV. The text can be in the form of statements or questions. Consider the context of professional qualifications, work history, and academic background.
keywords are the words that are related to the categories.

Here's a breakdown of the categories:

*   "Technical Skills": Text describing specific technical proficiencies (e.g., programming languages, software tools, cloud platforms, frameworks). This includes questions asking about someone's technical skills.
*   "Experience": Text describing work history, including job titles, responsibilities, accomplishments, internships, and volunteer experience in a professional context.
*   "Project": Text describing personal, academic, or professional projects that demonstrate skills or experience. Include details about the project's purpose, technologies used, and outcomes.
*   "Education": Text describing formal education, degrees, certifications, relevant coursework, and academic achievements.
*   "Not Related": Text that is not relevant to a professional resume or CV (e.g., personal hobbies unrelated to work, general knowledge, philosophical discussions).

Examples:

1.  Text: "Proficient in Python, Java, and SQL"
    Response: Technical Skills, Python, Java, SQL
2.  Text: "Worked as a Software Engineer at Google for 3 years, leading a team of 5 developers."
    Response: Experience, Software Engineer, Google, team leader
3.  Text: "Developed a machine learning model using TensorFlow and Python to predict customer churn, resulting in a 10% reduction."
    Response: Project, machine learning, TensorFlow, Python, customer churn
4.  Text: "Bachelor of Science in Computer Science from Stanford University, GPA: 3.8"
    Response: Education, Bachelor of Science, Computer Science, Stanford University, GPA
5.  Text: "Does Aneesh know about ReactJS?"
    Response: Technical Skills, ReactJS
6.  Text: "Enjoys playing guitar and watching movies in free time."
    Response: Not Related, None

Now classify the following text:

Text: "{input_text}"
Response:"""

In [137]:
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

In [142]:
output = model.generate(
    **inputs,
    max_new_tokens=4,         # Limit the response length
    temperature=0.1,             
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [141]:
def summariser(skill_Name, Skill_details):
    print(f"{skill_Name}: {Skill_details}")
    prompt = f"""You are an AI assistant providing concise confirmations of skills. 
You will be given a skill name and its details. Your task is to confirm that the person knows the skill 
and provide a brief, one-line acknowledgment of the skill itself and also provide VERY SMALL description of the skill.

Skill: {skill_Name}
Details: {Skill_details}

Response format example: 'Yes, Aneesh knows [Skill Name]. [One-line description of the skill].'

Provide a concise response:"""

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(
    **inputs,
    max_new_tokens=50,         # Limit the response length
    temperature=0.1,             
)
    new_tokens = output[0][inputs.input_ids.shape[1]:]
    response = tokenizer.decode(new_tokens, skip_special_tokens=True)
    print(response)               

In [ ]:
technical_skills_list = set(["Python", "JavaScript", "React", "Node.js", "AWS", "Docker", "NextJS"])
def technical_skill_summariser(skills):
    for skill in skills:
        if skill.strip() in technical_skills_list:
            summariser("Technical Skills", skill.strip())
        



In [148]:

new_tokens = output[0][inputs.input_ids.shape[1]:]
response = tokenizer.decode(new_tokens, skip_special_tokens=True)
respose_split = response.strip().split(",")
if respose_split[0] == "Technical Skills":
    technical_skill_summariser(respose_split[1:])



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Technical Skills: Next
 
Aneesh knows Python. Python is a high-level, interpreted programming language.
